| | |
|---|---|
| **Name** | Jan Ubbo van Baardewijk & Thomas Brus |
| **Group** | ML_HMI_01 |


# Lab 3: Training and testing

In this lab we deal with overfitting and generalisation. We illustrate how these occur by extending last week's lab which was implementing logistic regression using basis functions.

# Logistic regression with base functions

$\newcommand{\x}{\mathbf{x}}\newcommand{\w}{\mathbf{w}}\newcommand{\c}{\mathcal{C}}\newcommand{\feat}{\boldsymbol{\phi}}$

As we have seen last week, the logistic regression models the probability of a class label $\c$ given a datapoint $p(\c|\x)$ as:
$$p(\c|\x) = \sigma(\w^\top\x) = \frac{1}{1+e^{-\w^\top\x}}$$

In this lab, we shall use basis functions to make this model non-linear in $\x$, while still remaining linear in $\w$, by replacing $\x$ with a fixed, non-linear feature vector function $\feat = f(\x)$, so that $p(\c|\x)=\sigma(\w^\top\feat)$

## Data

We start by generating a dataset. As we have seen in the lecture, the central problem of machine learning is to devise algorithms that will generalise to previously unseen data, *i.e.*, that will correctly consider the distribution of the data-generating process into account, rather than merely the distribution of the training data. As you can imagine, this is more difficult for small training sets. The dataset in "data-2class-nonlin" is very similar to the data that we have used before but smaller and, therefore, harder.


In [4]:
# This code is given
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
import copy
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

maxexp=40 # Floats cannot represent exp(maxexp) and larger

def sigma(a):
    '''Logistic function, avoiding warnings of numerical overflows'''
    if a>maxexp:
        return 1.
    if a<-maxexp:
        return 0.
    return 1./(1.+np.exp(-a))
def log1pe(a):
    '''Numerically accurate implementation of log(1+e^x)'''
    if a<-maxexp:
        return 0.
    if a>maxexp:
        return a
    return np.log(1.+np.exp(a))
def logsigma(a):
    '''Numerically accurate log(sigma(x))'''
    return -log1pe(-a)


In [ ]:
# This code is given 


data = np.load("data-2class-nonlin.npz")
d = data['d']
l = data['l']

colours = [ [ 'b', 'r' ][int(x)] for x in np.nditer(l)] 

plt.scatter(d[:,0],d[:,1],20,colours)
plt.show()




# Basis functions

From the plot, it seems that we can do better than simply use a linear classifier. How about using a polynomial function as a discriminant? What about a quadratic function? Or a third-order polynomial?

Such a polynomial function is a function of the type $y = w_0 + w_1 x + w_2 x^2 + w_3 x^3$, *etc.*. Notice how, in the previous equation, y is a non-linear function of x _but a linear function of $\w$_. Optimising the function with respect to $\w$ is still just as easy as last week. In actual fact, we are interested in a function of $x_1$ and $x_2$ rather than only $x$, but the principle is the same: $f(\x,\w)$ is still a linear function of $\w$. 

**Question 1 [5 credits]**: Implement a Python function that takes in $\x$ and the order of the polynomial, and returns the feature vector $\feat$. For example `feat([x1,x2],2)` returns `[1,x1,x2,x1*x1,x1*x2,x2*x2]`. Bonus points for not including both `x1*x2` *and* `x2*x1` in the answer ;-).


In [5]:
#### Your answer to Q1 comes here


In [5]:
# This code is given
# Have an array storing the dimensionality of the different models

dims = [ len(feat([0,0],o)) for o in range(20)]
print dims 

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55, 66, 78, 91, 105, 120, 136, 153, 171, 190, 210]


## Gradient descent and plotting the probability 

Here is some code for visualising the probability of the class given the input and for performing the gradient descent on the weights.

In [ ]:
# This code is given

xx,yy = np.mgrid[-5:10:.1,-5:10:.1]

def plotHeat(w, order):
    p = np.zeros(xx.shape)
    for i in range(xx.shape[0]):
        for j in range(xx.shape[1]):
            p[i,j] = sigma(w.dot(feat(np.array([xx[i,j], yy[i,j]]), order)))
    plt.pcolor(xx,yy,p,cmap='seismic')
    plt.xlim([-5,10])
    plt.ylim([-5,10])
    plt.title("Order "+str(order))

def plotData():
    plt.scatter(d[:,0],d[:,1],20,colours)
    
w = np.array([0,1,1,1,1,1])
plotHeat(w,2)
plotData()



In [9]:
# This code is given

def errorfun(w, data, labels):
    '''Numerically accurate implementation of the error function'''
    logprob = 0
    for x,c in zip(data,labels):
        logprob += log1pe(-w.dot(x)) + (1.-c)*w.dot(x)

    return logprob

def optimiseDisc(w, data, labels, order, eta=1e-3, maxrun=10000):
    '''Implementation of batch gradient descent with adaptive step size'''
    newError = errorfun(w,data,labels)
    pastError = newError + 10
    es = []
    n=0
    while pastError-newError > 1e-5 and n<maxrun:
        n+=1
        grad = np.zeros(w.shape)
        for x,c in zip(data,labels):
            grad += (sigma(w.dot(x)) - c) * x
        pw = copy.copy(w)
        w -= eta * grad
        
        pastError = newError
        newError = errorfun(w,data,labels)
        while eta > 0. and pastError - newError < 0: # If the error increases, eta is too large
            eta /= 20                                # halve it, and
            w = pw - eta * grad                      # try again from the original value of the weights
            newError = errorfun(w,data,labels)
        else:                                        # If the error goes down,
            eta *= 1.2                               # try to increase eta a little, to speed up things.
        
        if newError<pastError:
            es.append(newError)
        else:                                        # If we couldn't decrease the error anymore, 
            break                                    # just give up.
#        print n, pastError, newError, pastError-newError, eta

    return w, newError
    


In [6]:
# Playground for using gradient descent...

# Generalisation

Evaluate the error on the training set of gradient descent, with different orders of your feature vector. Pay attention to the evolution of the error function during the training; if you notice that the error goes up at times, your step size $\eta$ is too large and you should reduce it. This error should keep diminishing: if it doesn't, you either stopped training too soon, or (for higher-order functions) you got stuck in a local optimum.

Local optima are a problem for complex models. Sometimes, a complex model really is warranted, but finding optimal parameter values for it is difficult. In general, one solution is to initialise the parameters at random, run multiple optimisation runs, and keep the run with the best results. 

**Question 2 [10 credits]**: In this case, where there is a clear relationship between the parameters of the models of increasing complexity, can you think of a way to ensure that increasing the complexity of the model is guaranteed to reduce the training error (and so that, even if you get stuck in a local optimum, you're at least certain that it is an optimum that's better than the best model you've found so far? Does this strategy have disadvantages?


*Your answer to Q2*
* ...


**Question 3 [10 credits]**: Divide your data into a training and test set. Plot the evolution of the error on both the train and the test set in function of the order of the polynomial.


In [9]:
# Your answer to Q3 comes here



**Question 4 [10 credits]**: Now perform 5-fold cross-validation on the data set and create the same plot as in the previous question. Doing cross-validation gives you a more accurate and stable estimate of the error of your model on unseen data, but it also gives you an idea of the spread of this error and therefore a (optimistic) idea of the upper and lower bounds on your error. Use error bars to show the standard deviation on your cross-validation error.



In [8]:
# Your answer to Q4 comes here

**Question 5 [5 credits]**: What order should the polynomial be, do you think? Why do you think that?

*Your answer to Q5*
